<a href="https://colab.research.google.com/github/Shayshu-NR/Notebook-Analysis/blob/main/PCard/Toronto_PCard_Expenditures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The below notebook looks into the expenditure of different municipal services in Toronto.

The main point of this is to view discrepancies, as well as see trends and where spending is allocated.

#  Data collection

In [1]:
import requests
import zipfile


# Toronto Open Data is stored in a CKAN instance. It's APIs are documented here:
# https://docs.ckan.org/en/latest/api/

# To hit our API, you'll be making requests to:
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"

# Datasets are called "packages". Each package can contain many "resources"
# To retrieve the metadata for this package and its resources, use the package name in this page's URL:
url = base_url + "/api/3/action/package_show"
params = { "id": "pcard-expenditures"}
package = requests.get(url, params = params).json()

# To get resource data:
for idx, resource in enumerate(package["result"]["resources"]):

       # To get metadata for non datastore_active resources:
       if not resource["datastore_active"]:
          url = base_url + "/api/3/action/resource_show?id=" + resource["id"]
          resource_metadata = requests.get(url).json()
          # From here, you can use the "url" attribute to download this file

          if resource_metadata["result"]["format"] == 'ZIP':
            response = requests.get(resource_metadata["result"]["url"])
            file_path = str(idx) + "_" + resource_metadata["result"]["name"] + ".zip"

            # Save zip and unzip
            with open(file_path, 'wb') as file:
              file.write(response.content)
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
              zip_ref.extractall("./")


# Data loading

In [2]:
file_path = "./expenditures/pcard-expenses/expenditures/"
from os import listdir
from os.path import isfile, join
expenditure_files = [f for f in listdir(file_path) if isfile(join(file_path, f))]

In [10]:
import numpy as np
import pandas as pd

df = pd.concat(pd.read_excel(file_path + f) for f in expenditure_files)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""C

In [9]:
# Drop empty rows which are used as sums of previous expenses
df_clean = df.dropna(subset='Division')
df_clean.head()

,Division,Batch-Transaction ID,Transaction Date,Card Posting Dt,Merchant Name,Transaction Amt.,Trx.Currency,Original Amount,Original Currency,G/L Account,...,Cost Centre / WBS Element / Order No.,Cost Centre / WBS Element / Order No. Decription,Cost Centre / WBS element / Order,Cost Centre / WBS element / Order Description,Transaction\n Date,Original\n Amount,Original\n Currency,Cost Centre / WBS\n Element,Unnamed: 16,Unnamed: 17
